# Logical Operators for CSS Codes

This notebook demonstrates how to set up a CSS encoder as a ZX-diagram and automatically check the validity of logical operators. We show this for (1) the logical CCZ gate for the [[8, 3, 2]] code, (2) the logical CZ gate for the [[8, 3, 2]] code, and (3) the logical T gate for the [[15, 1, 3]] code.

Input: 
- a CSS code (defined by its X-type stabilizers and logical operators)
- a set of single qubit gates applied (transversally) on physical qubits

Output:
- resulting graph after "pushing" the spiders representing physical gates towards the left to the logical side
- if no gate remains on stabilizers, then it is a valid logical operator, represented by the spider(s) to the left of the encoder

Note: Currently, the methods for pushing the physical gate to the left only work for single qubit Z-type physical gates (for example, it cannot check the transversal CNOT or transversal H gates for the Steane code), and the CSS encoder graph must be in Z-X normal form (defined by X-type stabilizers and logical operators).

This notebook uses the Graphical Fourier rewrites explained in [this notebook](H-box%20rewriting.ipynb).

In [ ]:
import pyzx as zx
from fractions import Fraction
from pyzx.rewrite_rules.basicrules import *
from pyzx.fourier import *
from pyzx.css import *
from pyzx.rewrite_rules.hrules import *
from pyzx.hsimplify import *
from pyzx.linalg import Mat2

In [2]:
Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
HB = zx.VertexType.H_BOX
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [3]:
def apply_single_qubit_physical_gates(g: Graph, vertex_indices: List[List[int]], gate_indices: List[int], gate: str="T"):
    """This function applies single qubit gates to the physical qubits of a CSS encoder graph.
        
        Arguments:
        g -- the stabilizer circuit
        vertex_indices -- a list of three lists of vertex indices
            logical_verts -- a list of indices of the logical vertices
            stabilizer_verts -- a list of indices of the stabilizer vertices
            output_verts -- a list of indices of the output vertices
        gate_indices -- a list of integers of length n (number of physical qubits), where at each index i:
            gate_indices[i] = 1 means the gate is applied to the i-th physical qubit, 
            gate_indices[i] = -1 means the gate^dagger is applied, and
            gate_indices[i] = 0 means no gate is applied
        gate -- the type of gate to apply (default is T)
       
       Returns:
        g -- the modified stabilizer circuit
        physical_gate_verts -- a list of vertex indices of the physical gate vertices
    """

    
    gate_type, gate_frac = None, None
    if gate == "T":
        gate_type = zx.VertexType.Z
        gate_phase = Fraction(1,4)
    elif gate == "S":
        gate_type = zx.VertexType.Z
        gate_phase = Fraction(1,2)

    logical_verts, stabilizer_verts, output_verts = vertex_indices
    physical_gate_verts = [-1 for i in range(len(gate_indices))]
    
    for i, gate_mode in enumerate(gate_indices):
        phase = gate_mode * gate_phase
        if gate_mode != 0:
            physical_gate_verts[i] = g.add_vertex(gate_type, qubit=i, row=7, phase=phase)
            output_vert = output_verts[i]
            out_bound_vert = [v for v in g.neighbors(output_vert) if g.type(v) == zx.VertexType.BOUNDARY][0]
            g.remove_edge((output_vert, out_bound_vert))
            g.add_edge((output_vert, physical_gate_verts[i]))
            g.add_edge((physical_gate_verts[i], out_bound_vert))
    return g, physical_gate_verts

In [4]:
def push_single_qubit_physical_gates_left(g, vertex_indices, gate_indices):    
    """This function pushes single qubit physical gates to the left of the encoder.
        Step 1: Unfuse the physical gate vertices, 
        Step 2: Apply the strong complementarity (SC) rule to push the gate vertices to the left of the encoder
        Step 3: Fuse the newly created vertices to the encoder vertices. The remaining gate phase vertice on the left form phase gadgets"""

    # We define a modified unfuse() function that can control the location of new vertex
    def unfuse(g, v, qubit=-1, row=-1):
        v_phase = g.phase(v)
        new_v = g.add_vertex(
            ty=Z,
            qubit=qubit,
            row=row,
            phase=v_phase)
    
        # unfuse the phase
        g.set_phase(v, phase=0)
        g.add_edge(g.edge(v, new_v), edgetype=SE)
    
        # return the reference to the new vertex
        return new_v
    
    n = len(gate_indices) # number of physical qubits
    unfused_physical_gate_vertices = [-1 for i in range(n)]
    logical_verts, stabilizer_verts, output_verts, physical_gate_verts = vertex_indices
    for qubit in range(n):
        # unfuse
        if gate_indices[qubit] != 0:
            unfused_physical_gate_vertices[qubit] = unfuse(g, physical_gate_verts[qubit], qubit-1, 9)
    phase_gadget_vertices = []

    # print("strong complementarity + fuse")
    for qubit in range(n):
        # sc push phase gadget through
        if gate_indices[qubit] == 0:
            continue
        output_vert = output_verts[qubit]
        physical_gate_vert = physical_gate_verts[qubit]
        left_Z_nei = [z_nei for z_nei in g.neighbors(output_vert) if g.type(z_nei) == Z and z_nei < output_vert]
        T_phase_Z = unfused_physical_gate_vertices[qubit]
        *_, last_id = g.vertices()
        strong_comp(g, output_vert, physical_gate_vert)
        
        for i, z in enumerate(left_Z_nei):
            fuse(g, z, last_id+i+1)
    
        Xspider1 = last_id+len(left_Z_nei)+1 # connected to boundary
        Xspider2 = Xspider1+1 # part of phase gadget, or remove_id
        g.set_position(Xspider1, qubit, 6)
    
        if not remove_id(g, Xspider2):
            g.set_position(Xspider2, qubit, -1)
            g.set_position(T_phase_Z, qubit, -2)
            phase_gadget_vertices.append((Xspider2, T_phase_Z))
        else:
            fuse(g, left_Z_nei[0], T_phase_Z)  # Xspider2 must have a single left Z spider neighbor
    return g, phase_gadget_vertices

In [5]:
# applies M rule, Eq 9.33
def simplify_hboxes(g):
    """This function applies the M rule and then H-box identity simplification rule (from Eq. 9.33 of PQS) to the encoder graph."""
    # apply M rule
    par_hbox_simp(g)

    # remove h-boxes with phase exponent 0. Ref: Eq 9.33 (from PQS)
    h_boxes = [v for v in g.vertices() if g.type(v) == HB]
    for hb in h_boxes:
        if g.phase(hb) == 0:
            g.remove_vertex(hb)

##  Example 1: Logical CCZ gate for [[8,2,3]] code

This shows that the logical CCZ gate is implemented transversally by applying the physical gates $$ T_1T_2^{\dagger}T^{\dagger}_3T_4T^{\dagger}_5T_6T_7T^{\dagger}_8 $$

Reference: https://earltcampbell.com/2016/09/26/the-smallest-interesting-colour-code/

First, for reference, we construct the target PyZX graph of the logical CCZ gate on the code, supported on logical wires only. This is what we want to get after pushing the physical gates to the left.


In [6]:
print("Target graph: logical CCZ gate for [[8,3,2]] code")
LX = Mat2([[1,1,1,1,0,0,0,0],[1,1,0,0,1,1,0,0],[1,0,1,0,1,0,1,0]])
SX = Mat2([[1,1,1,1,1,1,1,1]])
ccz_enc, _ = generate_css_encoder_graph(SX, LX, 'Z-X')
topz = ccz_enc.add_vertex(Z, qubit=0, row=1)
midz = ccz_enc.add_vertex(Z, qubit=1, row=1)
bottomz = ccz_enc.add_vertex(Z, qubit=2, row=1)
ccz_enc.remove_edge((0,1))
ccz_enc.remove_edge((2,3))
ccz_enc.remove_edge((4,5))
ccz_enc.add_edge((1,topz))
ccz_enc.add_edge((topz,0))
ccz_enc.add_edge((3,midz))
ccz_enc.add_edge((midz,2))
ccz_enc.add_edge((5,bottomz))
ccz_enc.add_edge((bottomz,4))
hbox = ccz_enc.add_vertex(HB, qubit=0.5, row=1.5)
ccz_enc.add_edge((hbox,topz))
ccz_enc.add_edge((hbox,midz))
ccz_enc.add_edge((hbox,bottomz))
zx.draw(ccz_enc, labels=True)

Target graph: logical CCZ gate for [[8,3,2]] code


Below is the code that demonstrates the equivalence of the transversal implementation of the CCZ gate and the logical gate itself for the [[8, 3, 2]] code.

In [7]:
print("STEP 0: Set up encoder graph for [[8, 3, 2]] code in Z-X normal form")
LX = Mat2([[1,1,1,1,0,0,0,0],[1,1,0,0,1,1,0,0],[1,0,1,0,1,0,1,0]])
SX = Mat2([[1,1,1,1,1,1,1,1]])
enc, vertex_ids = generate_css_encoder_graph(SX, LX, 'Z-X')
zx.draw(enc, labels=True)    # add transversal T gates to physical (right) side

print("STEP 1: Apply T gates on physical wires which implement the logical CCZ gate")
T_gates = [1,-1,-1,1,-1,1,1,-1]
enc, T_vertices = apply_single_qubit_physical_gates(enc, vertex_ids, T_gates, "T")
vertex_ids.append(T_vertices)
zx.draw(enc, labels=True)

STEP 0: Set up encoder graph for [[8, 3, 2]] code in Z-X normal form


STEP 1: Apply T gates on physical wires which implement the logical CCZ gate


Sanity check: verify that this graph (with physical implementation of logical CCZ) is equivalent to our target graph of the logical CCZ gate on logical wires

In [8]:
enc.auto_detect_io()
ccz_enc.auto_detect_io()
print(zx.compare_tensors(enc, ccz_enc, preserve_scalar=False))

True


Starting from the graph of the physical implementation, we apply the following step to rewrite it into the graph of the logical CCZ gate on logical wires.

In [9]:
# push T gates to left (logical side) of the encoder
print("STEP 2: Push T gates to the left side of encoder via SC")
enc, phase_gadget_v = push_single_qubit_physical_gates_left(enc, vertex_ids, T_gates)
zx.draw(enc, labels=True)

print("STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)")
for _, v in phase_gadget_v:
    ifourier(enc, v)
zx.draw(enc, labels=True)

print("STEP 4: Apply H-box simplification rules (M rule and 0 phase)")
simplify_hboxes(enc)
zx.draw(enc, labels=True)

STEP 2: Push T gates to the left side of encoder via SC


STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)


STEP 4: Apply H-box simplification rules (M rule and 0 phase)
par_hbox_simp: 9.  1 iterations


In [10]:
print(zx.compare_tensors(enc, ccz_enc, preserve_scalar=False))

True


We can see that this graph matches the target graph with the CCZ gate on logical wires (minus the fusing of the Z spiders of the CCZ gate with the Z spiders of the encoder). Therefore, this derivation verifies that the physical gates $ T_1T_2^{\dagger}T^{\dagger}_3T_4T^{\dagger}_5T_6T_7T^{\dagger}_8 $ indeed implement the logical CCZ gate for the [[8, 3, 2]] code.

##  Example 2: Logical CZ gate for [[8,2,3]] code

This shows that the logical CZ gate (applied to the first two qubits) for the [[8,2,3]] code is implemented transversally by applying the physical gates $$ S_2S_4^{\dagger}S_6^{\dagger}S_8 $$

Reference: 3b of https://www.math.uwaterloo.ca/~wcleung/a5-qeccw22.pdf

As before, we first set up the target graph of the logical CZ gate for reference.

In [11]:
# Construct target PyZX graph with CZ gate on logical wires only

print("Target graph: logical CZ gate for [[8,3,2]] code")
LX = Mat2([[1,1,1,1,0,0,0,0],[1,1,0,0,1,1,0,0],[1,0,1,0,1,0,1,0]])
SX = Mat2([[1,1,1,1,1,1,1,1]])
cz_enc, _ = generate_css_encoder_graph(SX, LX, 'Z-X')
topz = cz_enc.add_vertex(Z, qubit=0, row=1)
midz = cz_enc.add_vertex(Z, qubit=1, row=1)
cz_enc.remove_edge((0,1))
cz_enc.remove_edge((2,3))
cz_enc.add_edge((1,topz))
cz_enc.add_edge((topz,0))
cz_enc.add_edge((3,midz))
cz_enc.add_edge((midz,2))
hbox = cz_enc.add_vertex(HB, qubit=0.5, row=1.5)
cz_enc.add_edge((hbox,topz))
cz_enc.add_edge((hbox,midz))
zx.draw(cz_enc, labels=True)

Target graph: logical CZ gate for [[8,3,2]] code


In [12]:
print("STEP 0: Set up encoder graph for [[8, 3, 2]] code in Z-X normal form")
LX = Mat2([[1,1,1,1,0,0,0,0],[1,1,0,0,1,1,0,0],[1,0,1,0,1,0,1,0]])
SX = Mat2([[1,1,1,1,1,1,1,1]])
enc, vertex_ids = generate_css_encoder_graph(SX, LX, 'Z-X')
zx.draw(enc, labels=True)

print("STEP 1: Apply physical S gates implementing logical CZ on physical wires")
S_gates = [0,1,0,-1,0,-1,0,1]
enc, S_vertices = apply_single_qubit_physical_gates(enc, vertex_ids, S_gates, "S")
vertex_ids.append(S_vertices)
zx.draw(enc, labels=True)

print("Sanity check: verify that the graph with transversal CZ applied on physical wires = target graph with CZ on logical wires")
enc.auto_detect_io()
cz_enc.auto_detect_io()
print(zx.compare_tensors(enc, cz_enc, preserve_scalar=False))

print("STEP 2: Push S gates to the left side of encoder")
enc, phase_gadget_v = push_single_qubit_physical_gates_left(enc, vertex_ids, S_gates)
zx.draw(enc, labels=True)

print("STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)")
for _, v in phase_gadget_v:
    ifourier(enc, v)
zx.draw(enc, labels=True)

print("STEP 4: Apply H-box simplification rules (M rule and 0 phase)")
simplify_hboxes(enc)
zx.draw(enc, labels=True)

print("Final check: verify equivalence of derived encoder graph with target graph")
print(zx.compare_tensors(enc, cz_enc, preserve_scalar=False))

STEP 0: Set up encoder graph for [[8, 3, 2]] code in Z-X normal form


STEP 1: Apply physical S gates implementing logical CZ on physical wires


Sanity check: verify that the graph with transversal CZ applied on physical wires = target graph with CZ on logical wires
True
STEP 2: Push S gates to the left side of encoder


STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)


STEP 4: Apply H-box simplification rules (M rule and 0 phase)
par_hbox_simp: 2.  1 iterations


Final check: verify equivalence of derived encoder graph with target graph
True


We can see that this graph matches the target graph with the CZ gate on logical wires (minus the fusing of the Z spiders of the CZ gate with the Z spiders of the encoder). Therefore, this derivation verifies that the physical gates $ S_2S_4^{\dagger}S_6^{\dagger}S_8 $ indeed implement the logical CZ gate for the [[8, 3, 2]] code.

## Example 3: Logical T gate for [[15,1,3]] code

This shows that the logical T gate for the [[15, 1, 3]] code is implemented transversally by applying the physical gates $$(T^{\dagger})^ {\otimes 15}$$

In [13]:
print("STEP 0: Set up encoder graph for [[15, 1, 3]] code in Z-X normal form")
SX = Mat2([
    [1,0,1,0,1,0,1,0,1,0,1,0,1,0,1],
    [0,1,1,0,0,1,1,0,0,1,1,0,0,1,1],
    [0,0,0,1,1,1,1,0,0,0,0,1,1,1,1],
    [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]])
LX = Mat2([
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
enc, vertex_ids = generate_css_encoder_graph(SX, LX, 'Z-X')
zx.draw(enc, labels=True)

print("STEP 1: Apply physical T gates implementing the logical T gate, on physical wires")
T_gates = [-1 for _ in range(15)]
enc, T_vertices = apply_single_qubit_physical_gates(enc, vertex_ids, T_gates, "T")
vertex_ids.append(T_vertices)
zx.draw(enc, labels=True)

print("STEP 2: Push T gates to the left side of encoder via SC")
enc, phase_gadget_v = push_single_qubit_physical_gates_left(enc, vertex_ids, T_gates)
zx.draw(enc, labels=True)

print("STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)")
for _, v in phase_gadget_v:
    ifourier(enc, v)
zx.draw(enc, labels=True)

print("STEP 4: Apply H-box simplification rules (M rule and 0 phase)")
simplify_hboxes(enc)
zx.draw(enc, labels=True)

STEP 0: Set up encoder graph for [[15, 1, 3]] code in Z-X normal form


STEP 1: Apply physical T gates implementing the logical T gate, on physical wires


STEP 2: Push T gates to the left side of encoder via SC


STEP 3: Turn phase gadgets into H-boxes via Inverse Fourier Transform (Prop 3.7 of https://arxiv.org/pdf/1904.07551)


STEP 4: Apply H-box simplification rules (M rule and 0 phase)
par_hbox_simp: 24.  1 iterations


We can see that this graph shows a logical T gate applied to the [[15, 1, 3]] encoder (minus the fusing of the Z spiders of the CCZ gate with the Z spiders of the encoder). Therefore, this derivation verifies that applying the physical gates $(T^{\dagger})^ {\otimes 15}$ implements the logical T gate for the [[15, 1, 3]] code.